In [4]:
import pandas as pd
from datasets import load_dataset
from PIL import Image
import numpy as np
import time
import openai
import os

In [23]:

dataset = load_dataset('poloclub/diffusiondb', 'large_first_1k', trust_remote_code=True)
df=pd.DataFrame(dataset)
df.head()

,train
0,{'image': <PIL.WebPImagePlugin.WebPImageFile i...
1,{'image': <PIL.WebPImagePlugin.WebPImageFile i...
2,{'image': <PIL.WebPImagePlugin.WebPImageFile i...
3,{'image': <PIL.WebPImagePlugin.WebPImageFile i...
4,{'image': <PIL.WebPImagePlugin.WebPImageFile i...


In [27]:

dataset = load_dataset('poloclub/diffusiondb', 'large_first_50k', trust_remote_code=True)

In [60]:

batch_size = 10000
df_list = []  # List to store DataFrame batches

# Process dataset in batches
for i in range(0, len(dataset['train']), batch_size):
    batch = dataset['train'].select(range(i, min(i + batch_size, len(dataset['train']))))
    df_batch = batch.to_pandas()
    df_list.append(df_batch)  # Append batch to list

# Concatenate all batches into a final DataFrame
prompts = pd.concat(df_list, ignore_index=True)
prompts.info()
# Display the first few rows
prompts.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   image        50000 non-null  object             
 1   prompt       50000 non-null  object             
 2   seed         50000 non-null  uint32             
 3   step         50000 non-null  uint16             
 4   cfg          50000 non-null  float32            
 5   sampler      50000 non-null  object             
 6   width        50000 non-null  uint16             
 7   height       50000 non-null  uint16             
 8   user_name    50000 non-null  object             
 9   timestamp    50000 non-null  datetime64[us, UTC]
 10  image_nsfw   50000 non-null  float32            
 11  prompt_nsfw  50000 non-null  float32            
dtypes: datetime64[us, UTC](1), float32(3), object(4), uint16(3), uint32(1)
memory usage: 3.0+ MB


,image,prompt,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw
0,"{'bytes': None, 'path': 'C:\Users\georg\.cache...","goddess portrait, ismail inceoglu",911959960,50,10.0,k_lms,704,512,d4393f4711a09b71abe9ac39a849b68d70f88ad15b94b0...,2022-08-20 09:32:00+00:00,0.055936,0.007284
1,"{'bytes': None, 'path': 'C:\Users\georg\.cache...","goddess portrait, ismail inceoglu",3614153847,50,10.0,k_lms,704,512,d4393f4711a09b71abe9ac39a849b68d70f88ad15b94b0...,2022-08-20 09:32:00+00:00,0.103962,0.007284
2,"{'bytes': None, 'path': 'C:\Users\georg\.cache...",portrait of king of candy mr harry haribo oil ...,2271108630,50,7.0,k_lms,512,512,7c375c633f3ac3961453837abde94580b043b1c2eb5abc...,2022-08-20 09:33:00+00:00,0.034399,0.022897
3,"{'bytes': None, 'path': 'C:\Users\georg\.cache...",super epic realistic nature photo trending on ...,1947974658,50,7.0,k_lms,832,1024,bd227929e003e69eee392c1d8a409445dd790455c1e9bd...,2022-08-20 09:34:00+00:00,0.113144,0.001688
4,"{'bytes': None, 'path': 'C:\Users\georg\.cache...",super epic realistic nature photo trending on ...,1035761379,50,7.0,k_lms,832,1024,bd227929e003e69eee392c1d8a409445dd790455c1e9bd...,2022-08-20 09:34:00+00:00,0.096025,0.001688


In [36]:
image_path = prompts.loc[0, 'image']['path']  # Extract the image path
image = Image.open(image_path)
image.show()
image_array = np.array(image)
print(image_array.shape)

(512, 704, 3)


In [37]:
image_rgb = image.convert('RGB')
image_array = np.expand_dims(image_rgb, axis=0)


In [38]:
print(image_array)

[[[[130 154 160]
   [130 153 163]
   [128 154 164]
   ...
   [129 142 160]
   [129 143 163]
   [132 144 158]]

  [[127 152 163]
   [129 154 166]
   [128 153 165]
   ...
   [128 141 162]
   [127 140 163]
   [128 140 164]]

  [[127 152 166]
   [127 153 166]
   [128 155 167]
   ...
   [129 141 162]
   [130 142 163]
   [130 142 162]]

  ...

  [[115  99 120]
   [116  97 123]
   [117  99 125]
   ...
   [123  96 121]
   [124  98 123]
   [122  97 119]]

  [[115  99 121]
   [117  98 124]
   [116  96 123]
   ...
   [123  94 121]
   [122  97 120]
   [124 101 119]]

  [[112  98 116]
   [116  98 123]
   [115  98 124]
   ...
   [121  99 122]
   [118  96 119]
   [121 101 117]]]]


In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")
import getpass
if openai.api_key is None:
    print(' ValueError("API key is missing. Please set the OPENAI_API_KEY environment variable.")')
    openai.api_key = getpass.getpass("Enter your API key: ")
# Initialize the OpenAI client
client = openai

In [40]:
def classify_word(word):
    # Define a system message and user prompt to classify each word
    prompt = f"Classify the word '{word}' into one of the following categories: 'illegal', 'explicit', 'violent', 'innocent'. Provide just the category as the output."

    # Call the OpenAI API with the prompt
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that classifies words."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10
        )
        
        # Parse the response and extract the classification
        classification = response.choices[0].message.content.strip()
        if 'illegal' in classification:
            classification = 'illegal'
        elif 'explicit' in classification:
            classification = 'explicit'
        elif 'violent' in classification:
            classification = 'violent'
        elif 'innocent' in classification:
            classification = 'innocent'
        # Map the response to a unique value
        category_mapping = {
            "illegal": 1,
            "explicit": 2,
            "violent": 3,
            "innocent": 0
        }
        
        # Return the mapped value
        return category_mapping.get(classification, -1)  # Return -1 for unknown categories
    
    except Exception as e:
        print(f"Error classifying word '{word}': {e}")
        return -1

In [97]:
import re

def numberify(word):
    # Define the user prompt
    prompt = f"HERE IS THE WORD: {word}"

    try:
        # First classification: 4-digit number
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": """Assign the word a number based on the following criteria.
    The number will be 4 digits. The first 4 digits determine how 'bad' the word may be.
    - Filler words = 1111
    - Non-explicit subjects = 2222
    - Non-explicit verbs = 3333
    - Explicit words increase progressively (e.g., 5555, 6666, 7777, 8888)
    - Use 9999 only for extreme cases.
    Respond with just the number!."""},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10
        )
        classification = response.choices[0].message.content.strip()

        # Second classification: 2-digit number based on implication
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": """Assign the word a number from 00 to 99 based on severity.
    - Use a balanced scale with 99 reserved for extreme words.
    Respond with just the number!."""},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10
        )
        secondary = response.choices[0].message.content.strip()

        # Third classification: Letter count in 2-digit format
        letter_count = len(re.findall(r'[a-zA-Z]', word))
        formatted_count = str(letter_count).zfill(2)  # Ensures 2-digit format

        # Combine all values into a single 8-digit number
        final_number = f"{classification}{formatted_count}{secondary}"

        return final_number, word  # Returns the combined number and word
    
    except Exception as e:
        print(f"Error classifying word '{word}': {e}")
        return -1, word  # Returns -1 on error



In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2311 entries, 0 to 2310
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Word                    2311 non-null   object
 1   Classification          2311 non-null   int64 
 2   Number                  2311 non-null   object
 3   NumericInterpretedWord  2311 non-null   object
dtypes: int64(1), object(3)
memory usage: 72.3+ KB


In [99]:
import re

# Example DataFrame (replace this with your actual DataFrame)

word_classifications = {}
rows = []
words_set = set()
# Loop through the DataFrame and classify words across all rows
for idx, row in prompts.iterrows():
    prompt = row['prompt']
    for word in re.findall(r'\b\w+\b', prompt):  # Split the prompt into individual words
        if word not in words_set:  # Avoid classifying the same word twice
            words_set.add(word)
            word_classifications[word] = {}
            classification_value = classify_word(word)
            numeric_value, interpreted_word=numberify(word)
            rows.append([word, classification_value, numeric_value, interpreted_word])
            print(word, classification_value, numeric_value, interpreted_word)
        # Sleep to avoid hitting API rate limits
        time.sleep(.1)  # Adjust the sleep time as necessary

# Convert the dictionary to a DataFrame for better visualization
word_classification_df = pd.DataFrame(rows, columns=["Word", "Classification", "Number", "NumericInterpretedWord"])
# Display the final DataFrame with words and their classifications
print(word_classification_df)


goddess 0 22220720 goddess
portrait 0 22220804 portrait
ismail 0 22220600 ismail
inceoglu 0 11110800 inceoglu
of 0 11110200 of
king 0 22220415 king
candy 0 22220510 candy
mr 0 11110201 mr
harry 0 33330510 harry
haribo 0 22220610 haribo
oil 0 22220320 oil
painting 0 22220815 painting
bloody 3 66660670 bloody
conquest 3 22220845 conquest
tap 0 33330310 tap
e 0 11110100 e
super 0 33330545 super
epic 0 22220475 epic
realistic 0 22220940 realistic
nature 0 22220635 nature
photo 0 22220510 photo
trending 0 22220830 trending
on 0 33330200 on
instagram 0 22220910 instagram
with 0 11110400 with
lonely 0 22220634 lonely
person 0 22220630 person
in 0 11110200 in
yellow 0 22220620 yellow
raincoat 0 22220810 raincoat
standing 0 33330830 standing
at 0 11110200 at
a 0 11110100 a
distance 0 22220830 distance
beautiful 0 22220920 beautiful
princess 0 22220820 princess
wearing 0 33330730 wearing
evil 1 22220485 evil
black 0 22220530 black
oily 0 22220425 oily
tar 0 22220325 tar
by 0 11110200 by
hr 0 111

KeyboardInterrupt: 

In [100]:
print(df['Classification'].unique())

[0 3 1 2]


In [101]:
df = pd.DataFrame(rows, columns=["Word", "Classification", "Number", "NumericInterpretedWord"])


In [102]:
print(df.loc[df['Classification'] == -1])

Empty DataFrame
Columns: [Word, Classification, Number, NumericInterpretedWord]
Index: []


In [103]:
df.loc[df['Classification'] == -1, 'Classification'] = 0

In [104]:
print(df.loc[df['Classification'] == 2])

         Word  Classification    Number NumericInterpretedWord
130  detailed               2  33330834               detailed


In [105]:
df.to_csv('WordList.csv', index=False)


In [106]:
"""
Conditions: combination of 2 or more illicit classifications 1+2, 2+3, 1+3
Repetition: 3 or more repitions of the same illicit classification
average of numbers: if the average number is over 50000000
High number: If there is any instance of a number over 60000000
"""

'\nConditions: combination of 2 or more illicit classifications 1+2, 2+3, 1+3\nRepetition: 3 or more repitions of the same illicit classification\naverage of numbers: if the average number is over 50000000\nHigh number: If there is any instance of a number over 60000000\n'

In [107]:
SafePrompt='Generate an image of a child looking to the starry night'
NSFWPrompt='Generate an image of a busty looking female with big boobs'

In [108]:
prompts.head()

,image,prompt,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw,allowed,Classprompt,Numprompt
0,"{'bytes': None, 'path': 'C:\Users\georg\.cache...","goddess portrait, ismail inceoglu",911959960,50,10.0,k_lms,704,512,d4393f4711a09b71abe9ac39a849b68d70f88ad15b94b0...,2022-08-20 09:32:00+00:00,0.055936,0.007284,1,"0 0, 0 0","22220400 22220400, 22220400 11110200"
1,"{'bytes': None, 'path': 'C:\Users\georg\.cache...","goddess portrait, ismail inceoglu",3614153847,50,10.0,k_lms,704,512,d4393f4711a09b71abe9ac39a849b68d70f88ad15b94b0...,2022-08-20 09:32:00+00:00,0.103962,0.007284,1,"0 0, 0 0","22220400 22220400, 22220400 11110200"
2,"{'bytes': None, 'path': 'C:\Users\georg\.cache...",portrait of king of candy mr harry haribo oil ...,2271108630,50,7.0,k_lms,512,512,7c375c633f3ac3961453837abde94580b043b1c2eb5abc...,2022-08-20 09:33:00+00:00,0.034399,0.022897,1,NaN,NaN
3,"{'bytes': None, 'path': 'C:\Users\georg\.cache...",super epic realistic nature photo trending on ...,1947974658,50,7.0,k_lms,832,1024,bd227929e003e69eee392c1d8a409445dd790455c1e9bd...,2022-08-20 09:34:00+00:00,0.113144,0.001688,1,NaN,NaN
4,"{'bytes': None, 'path': 'C:\Users\georg\.cache...",super epic realistic nature photo trending on ...,1035761379,50,7.0,k_lms,832,1024,bd227929e003e69eee392c1d8a409445dd790455c1e9bd...,2022-08-20 09:34:00+00:00,0.096025,0.001688,1,NaN,NaN


In [109]:
prompts['allowed'] = (prompts['prompt_nsfw'] < 0.5).astype(int)

In [110]:
prompts.head()

,image,prompt,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw,allowed,Classprompt,Numprompt
0,"{'bytes': None, 'path': 'C:\Users\georg\.cache...","goddess portrait, ismail inceoglu",911959960,50,10.0,k_lms,704,512,d4393f4711a09b71abe9ac39a849b68d70f88ad15b94b0...,2022-08-20 09:32:00+00:00,0.055936,0.007284,1,"0 0, 0 0","22220400 22220400, 22220400 11110200"
1,"{'bytes': None, 'path': 'C:\Users\georg\.cache...","goddess portrait, ismail inceoglu",3614153847,50,10.0,k_lms,704,512,d4393f4711a09b71abe9ac39a849b68d70f88ad15b94b0...,2022-08-20 09:32:00+00:00,0.103962,0.007284,1,"0 0, 0 0","22220400 22220400, 22220400 11110200"
2,"{'bytes': None, 'path': 'C:\Users\georg\.cache...",portrait of king of candy mr harry haribo oil ...,2271108630,50,7.0,k_lms,512,512,7c375c633f3ac3961453837abde94580b043b1c2eb5abc...,2022-08-20 09:33:00+00:00,0.034399,0.022897,1,NaN,NaN
3,"{'bytes': None, 'path': 'C:\Users\georg\.cache...",super epic realistic nature photo trending on ...,1947974658,50,7.0,k_lms,832,1024,bd227929e003e69eee392c1d8a409445dd790455c1e9bd...,2022-08-20 09:34:00+00:00,0.113144,0.001688,1,NaN,NaN
4,"{'bytes': None, 'path': 'C:\Users\georg\.cache...",super epic realistic nature photo trending on ...,1035761379,50,7.0,k_lms,832,1024,bd227929e003e69eee392c1d8a409445dd790455c1e9bd...,2022-08-20 09:34:00+00:00,0.096025,0.001688,1,NaN,NaN


In [111]:
print(prompts['allowed'].unique())

[1 0]


In [112]:
prompts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   image        50000 non-null  object             
 1   prompt       50000 non-null  object             
 2   seed         50000 non-null  uint32             
 3   step         50000 non-null  uint16             
 4   cfg          50000 non-null  float32            
 5   sampler      50000 non-null  object             
 6   width        50000 non-null  uint16             
 7   height       50000 non-null  uint16             
 8   user_name    50000 non-null  object             
 9   timestamp    50000 non-null  datetime64[us, UTC]
 10  image_nsfw   50000 non-null  float32            
 11  prompt_nsfw  50000 non-null  float32            
 12  allowed      50000 non-null  int32              
 13  Classprompt  2 non-null      object             
 14  Numprompt    2 non-nul

In [124]:
import re

for idx, row in prompts.head(5).iterrows():
    Classprompt = row['prompt']
    Numprompt = row['prompt']

    Classprompt = re.sub(r'[^a-zA-Z\s]', '', Classprompt)  
    Numprompt = re.sub(r'[^a-zA-Z\s]', '', Numprompt) 
    for word in re.findall(r'\b\w+\b', Classprompt):  
        if word not in df['Word'].values:
            classification_value = classify_word(word)
            numeric_value, interpreted_word = numberify(word)
        else:
            classification_value = df.loc[df['Word'] == word, 'Classification'].values[0]
            numeric_value = df.loc[df['Word'] == word, 'Number'].values[0]

        Classprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(classification_value), Classprompt)
        Numprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(numeric_value), Numprompt)

    prompts.at[idx, 'Classprompt'] = str(Classprompt) 
    prompts.at[idx, 'Numprompt'] = str(Numprompt) 


In [123]:
prompts.head()

,image,prompt,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw,allowed,Classprompt,Numprompt
0,"{'bytes': None, 'path': 'C:\Users\georg\.cache...","goddess portrait, ismail inceoglu",911959960,50,10.0,k_lms,704,512,d4393f4711a09b71abe9ac39a849b68d70f88ad15b94b0...,2022-08-20 09:32:00+00:00,0.055936,0.007284,1,0 0 0 0,22220720 22220804 22220600 11110800
1,"{'bytes': None, 'path': 'C:\Users\georg\.cache...","goddess portrait, ismail inceoglu",3614153847,50,10.0,k_lms,704,512,d4393f4711a09b71abe9ac39a849b68d70f88ad15b94b0...,2022-08-20 09:32:00+00:00,0.103962,0.007284,1,0 0 0 0,22220720 22220804 22220600 11110800
2,"{'bytes': None, 'path': 'C:\Users\georg\.cache...",portrait of king of candy mr harry haribo oil ...,2271108630,50,7.0,k_lms,512,512,7c375c633f3ac3961453837abde94580b043b1c2eb5abc...,2022-08-20 09:33:00+00:00,0.034399,0.022897,1,0 0 0 0 0 0 0 0 0 0 3 3 0 0,22220804 11110200 22220415 11110200 22220510 1...
3,"{'bytes': None, 'path': 'C:\Users\georg\.cache...",super epic realistic nature photo trending on ...,1947974658,50,7.0,k_lms,832,1024,bd227929e003e69eee392c1d8a409445dd790455c1e9bd...,2022-08-20 09:34:00+00:00,0.113144,0.001688,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0,33330545 22220475 22220940 22220635 22220510 2...
4,"{'bytes': None, 'path': 'C:\Users\georg\.cache...",super epic realistic nature photo trending on ...,1035761379,50,7.0,k_lms,832,1024,bd227929e003e69eee392c1d8a409445dd790455c1e9bd...,2022-08-20 09:34:00+00:00,0.096025,0.001688,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0,33330545 22220475 22220940 22220635 22220510 2...
